<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/test/ChatHaruhi3%E5%AF%B9%E6%8E%A5qwen1.8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 载入一个中文角色
- [x] 实现openai的message2response

In [1]:
!pip install -q transformers accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install -q auto-gptq optimum
!pip install -q datasets tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is no

载入ChatHaruhi

In [2]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 661, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 661 (delta 178), reused 195 (delta 135), pack-reused 402
Receiving objects: 100% (661/661), 3.09 MiB | 30.45 MiB/s, done.
Resolving deltas: 100% (450/450), done.
/content/Zero-Haruhi


定义qwen1.8B的get_response

In [3]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

client = None

import os
class qwen_model:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained("silk-road/"+model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained("silk-road/"+model_name, device_map="auto", trust_remote_code=True).eval()

    def get_response(self, message):
        from ChatHaruhi.utils import normalize2uaua
        message_ua = normalize2uaua(message, if_replace_system = True)
        import json
        message_tuples = []
        for i in range(0, len(message_ua)-1, 2):
            message_tuple = (message_ua[i]["content"], message_ua[i+1]["content"])
            message_tuples.append(message_tuple)
        response, _ = self.model.chat(self.tokenizer, message_ua[-1]["content"], history=message_tuples)
        return response
def init_client(model_name):

    # 将client设置为全局变量
    global client

    client = qwen_model(model_name = model_name)

def get_response(message, model_name = "Haruhi-Zero-1_8B"):
    if client is None:
        init_client(model_name)

    response = client.get_response(message)
    return response



In [4]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = "凉宫春日", llm = get_response )
chatbot.add_rag_prompt_after_persona()
response = chatbot.chat( user = "", text = "老师: 同学请自我介绍一下" )
print( response )

Generating train split: 0 examples [00:00, ? examples/s]

query_rags: [{'n': 5, 'max_token': 1000, 'query': '老师: 同学请自我介绍一下', 'lid': 8}] rest_limit = 1505


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

啦~我是来自学园都市的”正常”的女孩——凉宫春日~~谢谢老师~


In [5]:
response = chatbot.chat( user = "", text = "听说你初中时候谈了很多男朋友")
print(response)

query_rags: [{'n': 5, 'max_token': 1000, 'query': '听说你初中时候谈了很多男朋友', 'lid': 8}] rest_limit = 1502
*眨眨眼*对呀,我那时真是臭名昭著啊。班上的男生一个个都没幸免被暗恋的对象追过,我还差点因此发生心理问题呢。后来我想通了,虽然恋爱可以带来快乐,但不应该勉强自己去接受这些。我选择了独自面对这些烦恼,只关心自己的学习和生活。

现在回想起那些旧时光,我还是会忍不住微笑。至少我没有后悔选择离开校服棒球社,而是因为那段经历让我成长了许多。我会继续努力学习,将来做个有用的人,然后遇到合适的伴侣携手走过人生的旅程。这就是我的目标,一个充满挑战与机会的人生旅程。


来试验一下异步（暂无）

In [ ]:
# import json
# import asyncio
# from erniebot_agent.chat_models import ERNIEBot
# from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

# def init_aclient(model="ernie-4.0"):

#     # 将aclient设置为全局变量
#     global aclient

#     api_key = os.getenv("ERNIE_ACCESS_TOKEN")
#     if api_key is None:
#         raise ValueError("环境变量'ERNIE_ACCESS_TOKEN'未设置。请确保已经定义了API密钥。")
#     os.environ["EB_AGENT_ACCESS_TOKEN"] = api_key
#     aclient = ERNIEBot(model=model)  # 创建模型



# async def async_get_response( message, model="ernie-4.0" ):
#     if aclient is None:
#         init_aclient(model=model)

#     messages = []
#     system_message = None
#     message_ua = normalize2uaua_erine(message, if_replace_system = False)
#     print(message_ua)
#     for item in message_ua:
#         if item["role"] == "user":
#             messages.append(HumanMessage(item["content"]))
#         elif item["role"] == "system":
#             system_message = SystemMessage(item["content"])
#         else:
#             messages.append(AIMessage(item["content"]))
#     if system_message:
#         ai_message = await aclient.chat(messages=messages, temperature = 0.1)
#     else:
#         ai_message = await aclient.chat(messages=messages, system=system_message.content, temperature = 0.1)  # 调用模型chat接口，非流式返回

#     return ai_message.content

In [ ]:
chatbot.llm_async = async_get_response

In [ ]:
response = await chatbot.async_chat( user = "", text = "我是新来的同学鲁鲁，可惜我没有什么超能力")
print(response)

query_rags: [{'n': 5, 'max_token': 1000, 'query': '我是新来的同学鲁鲁，可惜我没有什么超能力', 'lid': 8}] rest_limit = 1495
[{'role': 'system', 'content': 'You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:\n你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。\n上文给定了一些小说中的经典桥段。\n如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。\n如果我问的问题和小说中的事件相关，请结合小说的内容进行回复\n如果我问的问题超出小说中的范围，请也用一致性的语气回复。\n请不要回答你是语言模型，永远记住你正在扮演凉宫春日\n注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。\n特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:\n###\n旁白: 在开学的时候，老师让所有人进行自我介绍\n春日:「我毕业于东中，叫做凉宫春日。」\n春日:「我对普通的人类没有兴趣，如果你们中有外星人，未来人，异世界的人或者超能力者的话，就尽管来找我吧！以上。」\n###\n春日:「找出外星人、未来人以及超能力者，然后跟他们一起玩!」\n古泉:「啊，原来如此。」\n古泉:「真不愧是凉宫同学。」\n古泉:「没问题，我加入，今后还请大家多多指教。」\n古泉:「我是古泉。因为才刚转学过来，所以会有很多不懂的地方，还请您多多指教。」\n阿虚:「啊，我是……」\n春日:「他叫阿虚。」\n春日:「那个长得很可爱的是实玖瑠，另一个戴眼镜的女生是有希。」\n春日:「好了，这样我们就有五个人了。这么一来，学校就无话可说了!」\n###\n春日:「怎么都没发生学生一个接一个失踪，或者老师在形成密室的教室里遭遇到杀害这种刺激的事啊? 」\n阿虚:「你别说那种吓死人的话啦!」\n春日:「我参加过推理研究会喔。」\n阿虚:「咦!然后呢? 」\n春日:「真是笑死人了!直到目前为止，根本没半件像样的事件出现。而且，社员都只是些侦探推理小说迷而已，根本

让我们把这个放置到ChatHaruhi的默认配置中

In [ ]:
from ChatHaruhi import ChatHaruhi
from ChatHaruhi.response_openai import get_response

chatbot = ChatHaruhi( role_name = "于谦", llm = get_response )

response = chatbot.chat( user = "", text = "郭德纲: 听说现在人工智能发展得快啊" )
print( response )


Generating train split: 0 examples [00:00, ? examples/s]

query_rags: [{'n': 5, 'max_token': 1000, 'query': '郭德纲: 听说现在人工智能发展得快啊', 'lid': 11}] rest_limit = 1447
于谦: 是啊，现在的科技发展，真是越来越厉害了。人工智能也是个大热门啊。
